# Introdução a Blockchain

Uma blockchain é extamente o que o nome diz: uma rede de blocos (blocos são conjuntos de transações). Essas redes tiveram uma adoção muito alta por parte da comunidade por causa de três grandes pontos: **imutabilidade**, **descentralização** e **privacidade**.

## Imutabilidade
> A imutabilidade é uma das principais, se não a principal, característica de uma blockchain. Como a rede é arquitetada em uma sequência de blocos, todas as informações estão conectadas, dessa forma, qualquer mudança (mesmo que apenas uma letra), será facilmente detectada, pois boa parte da rede irá mudar.

> Imagine o seguinte cenário: Fernando, uma pessoa má intencionada, tenta alterar a rede, ele muda uma transação passada para que ganhe mais dinheiro. Caso a rede não tivesse essa característica de imutabilidade, ele poderia simplesmente fazer isso e dificilmente alguem perceberia. Contudo, essa mudança irá mudar e ao mesmo tempo invalidar (depois será explicado melhor, por enquanto aceitem a informação) **todos** os blocos que foram adicionados depois do bloco adulterado por Fernando, assim as outras partes atuantes poderão impedi-lo imediatamente. 

## Descentralização
> A descentralização é outra característica crucial de uma rede blockchain. Em suma, uma blockchian é uma rede distribuida em diversos computadores, ou seja, ao contrário de uma base de dados tradicional que guarda toda sua informação em um único servidor, para que uma blockchain funcione é crucial que diversos (quanto mais melhor) computadores armazenem todos os blocos.

>Para exemplificar vou voltar ao exemplo anterior: caso a blockchain que Fernando utiliza fosse armazenada em um único servidor, se o mesmo tivesse acesso a esta base de dados, ele poderia mudar a transação e alterar todos os blocos em sequência para torna-los validos. Dessa forma sua mudança será dificilmente perceptível. Porém, como uma blockchain é armazenada em diversos computadores, para que Fernando consiga alterar a rede, seria necessário mudar 50% de todos os computadores que armazenam a rede ao mesmo tempo, o que é praticamente impossível.

## Privacidade

> A última grande característica de uma blockchain é a privacidade. Em geral um usuário na rede é representado por um endereço (uma sequência única de números e letras), e não pelo nome/identidade. Vale notar que você pode ter quantos endereços quiser.

> No exemplo anterior, após Fernando tentar adulterar a rede em seu favor, caso ele fosse representado pela sua identidade de pessoa física, os integrantes da rede poderiam escolher excluí-lo da rede, contudo, como ninguêm sabe seu endereço, ele pode continuar usando a blockchain anonimamente.

## Transações e Blocos

Beleza, uma blockchain é um rede descentralizada, imutável e os usuários são anônimos. Além disso, a rede é formada pela ligação de blocos, mas o que são os blocos?

 **Blocos**
> Como mencionado anteriormente blocos são conjuntos de transações, e isso é o que diferencia um bloco do outro: para formar um bloco é necessário escolher todas as transações que irão fazer parte do mesmo e calcular um identificador único para o mesmo - isso que é a famosa mineração de crypto. Esse identificador na verdade se chama hash, e ele é gerado por uma funçao que recebe algo e calcula um output único, por exemplo, as palavras caneta e canetas teriam outputs (hashes) completamente diferentes.

> Para formar e calcular o hash de um bloco, é necessário de algumas informações:
> * As transações que irão fazer parte de um bloco (em formato de [merkle root](https://www.investopedia.com/terms/m/merkle-root-cryptocurrency.asp#:~:text=A%20Merkle%20root%20is%20a,whole%2C%20undamaged%2C%20and%20unaltered), não entrarei em detalhes nesse momento)
> * O a data que o bloco será minerado em Unix (tempo medido em segundos após 1970)
> * O hash do bloco anterior (ai que é feita a ligação entre os blocos, qualquer mudança em um bloco anterior alterará o hash de todos os blocos posteriores)
> * A altura do bloco (número de blocos antes do mesmo)
> * Por último, duas informações que são o core da mineração: a **dificuldade da rede** e o **"nounce"**

> **Dificuldade da rede e nounce**
>> Um exemplo de um hash é este: 127e6fbfe24a750e72930c220a8e138275656b8e5d8f48a98c3c92df2caba935.


>>Ele é exatamente uma sequencia de números e letras aleatórias. A dificuldade da rede é diretamente ligada ao hash que os blocos tem que ter. Uma dificuldade de 2 significa que o hash tem que ter dois zeros no começo, 3 três zeros, e assim em diante. Mas se a funçao que produz o hash, quando recebe a mesma coisa (por exeplo maça sempre terá o mesmo hash), sempre retornará o mesmo hash, como que os mineradores vão mudar o hash do bloco para que tenha x zeros no começo? Uma opcão seria mudar todas as transações até chegar no resultado desejado, contudo, isso seria muito lerdo e complexo. Por esse motivo foi criado o nounce. O **Nounce** é um campo que é preenchido por um número, dessa forma, para mudar o hash, é apenas necessário mudar o número do nounce. Assim minerar transações na Bitcoin é literalmente juntar todas essas funções e ficar mudando o nounce até que o hash tenha a quantidade de zeros inicias referentes a dificuldade da rede.

## Imagem ilustrativa de blocos em uma rede blockchain

![rede blockchain](img/blockchain.png "Exemplo rede blockchain")

# Zezo faz aqui o handout

In [1]:
import time
from Crypto import Hash
import requests
from hashlib import sha256
from merkle import calculate_merkle_root


#new_post = requests.post("https://blockchainsper.herokuapp.com/mine", json= block)

#print(new_post.text)

In [2]:
timestamp = time.time()

info = requests.get("https://blockchainsper.herokuapp.com/info").json()

difficulty = info["difficulty"]
height = info["height"]
last_hash = info["last_hash"]
valid = info["valid"]

In [3]:
def new_hash(difficulty, height, last_hash, timestamp, nounce, message):
    merkle_root = calculate_merkle_root(message)
    header = "{}{}{}{}{}{}".format(height, last_hash, merkle_root, timestamp, difficulty, nounce)
    intermediary_hash = sha256(header.encode())
    real_hash = sha256(intermediary_hash.hexdigest().encode())
    return real_hash.hexdigest(), merkle_root

def new_block(difficulty, height, last_hash, timestamp, message):
    nounce = 0 
    hash_new = ""

    while hash_new[:difficulty] != "0" * difficulty:
        nounce += 1
        hash_new, merkle_root = new_hash(difficulty, height, last_hash, timestamp, nounce, message)

    new_block = {"difficulty":difficulty,"height":height,"merkle_root":merkle_root,"nonce": nounce,"previous_hash":last_hash,"timestamp":timestamp,"tx":message}
    return new_block

message = ["Blockchain Insper", "Tech 2021.2"]

block = new_block(difficulty, height, last_hash, timestamp, message)